In [10]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import os
import collections
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
######### Valence And Arousal
def SVMPCA(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ArousalEncoded'+".csv")
    ArousalFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceEncoded'+".csv")
    ValenceFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test , z_train,z_test= train_test_split(BandsFile, ValenceFile,ArousalFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    #Predict Arousal SVM and PCA
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
        
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    clf = SVC()
    clf.fit(X_train, z_train)
    z_predict = clf.predict(X_test)
    return [accuracy_score(z_test, z_predict)*100,accuracy_score(y_test, y_predict)*100
            ,f1_score(z_test, z_predict, average='weighted'),
            f1_score(y_test, y_predict, average='weighted')]

######### Happy And sad ##############
def SVMPCAHS(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'HappyAndSadEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'HS'+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    while ((np.sum(y_train == 0) == 0) or (np.sum(y_train == 1) == 0)): #Don't Start training unless you have 2 classes.. happens with imbalance
        print("here")
        X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]

######### All Emotions##############
def SVMPCACombined(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    file_path = os.path.join("ChannelsExtracted",UserName, 'ValenceAndArousalEncoded'+".csv")
    SurveyFile =  np.genfromtxt(fname=file_path,delimiter=',').astype(int)
    BandsFile = np.genfromtxt('training/'+str(StartTime)+str(EndTime)+'/'+str(Window)+UserName+'.csv',delimiter=',')
    X_train, X_test, y_train, y_test = train_test_split(BandsFile, SurveyFile, test_size=0.2)
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    # PCA to select features
    pca = PCA(n_components=Components, svd_solver='full')
    pca.fit(BandsFile)
    BandsFile = pca.transform(BandsFile)
    #Predict Valence SVM and PCA
    clf = SVC()
    clf.fit(X_train, y_train)
    y_predict = clf.predict(X_test)
    return [accuracy_score(y_test, y_predict)*100,
            f1_score(y_test, y_predict, average='weighted')]


############## Run Each Method Multiple Times ######################
def Run100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    arousal=[]
    valence=[]
    F1_ScoreArousal=[]
    F1_ScoreValence=[]
    for i in range (100):
        svm=SVMPCA(UserName,Components,Window,StartTime,EndTime)
        arousal+=[svm[0]]
        valence+=[svm[1]]
        F1_ScoreArousal+=[svm[2]]
        F1_ScoreValence+=[svm[3]]
    print("Arousal "+str(np.mean(arousal)))
    print(collections.Counter(arousal))
    print("Valence "+str(np.mean(valence)))
    print(collections.Counter(valence))
    return [np.mean(arousal),np.mean(valence),np.mean(F1_ScoreArousal),np.mean(F1_ScoreValence)];

def RunCombined100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_ALL=[]
    for i in range (100):
        AllEmotions=SVMPCACombined(UserName,Components,Window,StartTime,EndTime)
        Combined+=[AllEmotions[0]]
        F1_ALL+=[AllEmotions[1]]
    print("Array of average All Emotions accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_ALL)];
    
def RunHS100(UserName,Components=10,Window=256,StartTime=0,EndTime=63):
    Combined=[]
    F1_HS=[]
    for i in range (100):
        HS=SVMPCAHS(UserName,Components,Window,StartTime,EndTime)
        Combined+=[HS[0]]
        F1_HS+=[HS[1]]
    print("Array of average Happy Sad accurecies for user "+UserName)
    print(collections.Counter(Combined))
    return [np.mean(Combined),np.mean(F1_HS)];
        

    
##################Main#########################33
if __name__ == "__main__":
    HSAccurecy=0;
    HSF1=0;
    CombinedAccurecy=0;
    CombinedF1=0;
    ValenceAccurecy=0;
    ValenceF1=0;
    ArousalAccurecy=0;
    ArousalF1=0;
    for i in range(1,10):
        print("s0"+str(i))
        #Happy and Sad
        var =RunHS100("s0"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s0"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s0"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
        ValenceF1+=var2[3]
    for i in range(10,31):
        print("s"+str(i))
        #Happy and Sad
        var =RunHS100("s"+str(i))
        print("Happy Sad")
        print(var)
        HSAccurecy+=var[0]
        HSF1+=var[1]
        #All 4 Emotions
        var1 =RunCombined100("s"+str(i))
        print("Combined Emotions")
        print(var1)
        CombinedAccurecy+=var1[0]
        CombinedF1+=var1[1]
        #Seperated Valence and Arousal
        var2=Run100("s"+str(i))
        print("Arousal Valence")
        print(var2)
        ArousalAccurecy+=var2[0]
        ValenceAccurecy+=var2[1]
        ArousalF1+=var2[2]
    print("Accurecy happy sad for first 30 "+str(HSAccurecy/30))
    print("F1 happy sad for first 30 "+str(HSF1/30))
    print("Accurecy Combined sad for first 30 "+str(CombinedAccurecy/30))
    print("F1 Combined sad for first 30 "+str(CombinedF1/30))
    print("Accurecy Valence for first 30 "+str(ValenceAccurecy/30))
    print("F1 Valence for first 30 "+str(ValenceF1/30))
    print("Accurecy Arousal for first 30 "+str(ArousalAccurecy/30))
    print("F1 Arousal for first 30 "+str(ArousalF1/30))


s01
Array of average Happy Sad accurecies for user s01
Counter({40.0: 41, 60.0: 35, 80.0: 13, 20.0: 11})
Happy Sad
[50.0, 0.41065873015873017]
Array of average All Emotions accurecies for user s01
Counter({37.5: 33, 25.0: 31, 12.5: 18, 50.0: 14, 0.0: 2, 62.5: 2})
Combined Emotions
[30.625, 0.23304897186147192]
Arousal 56.0
Counter({62.5: 35, 50.0: 26, 37.5: 17, 75.0: 16, 25.0: 3, 12.5: 1, 100.0: 1, 87.5: 1})
Valence 36.0
Counter({37.5: 35, 25.0: 24, 50.0: 22, 12.5: 12, 62.5: 7})
Arousal Valence
[56.0, 36.0, 0.45412734487734485, 0.30878643578643583]
s02
Array of average Happy Sad accurecies for user s02
Counter({80.0: 43, 100.0: 34, 60.0: 20, 40.0: 3})
Happy Sad
[81.6, 0.7426349206349206]
Array of average All Emotions accurecies for user s02
Counter({50.0: 36, 37.5: 24, 62.5: 17, 25.0: 15, 12.5: 5, 75.0: 3})
Combined Emotions
[44.25, 0.3106877913752914]
Arousal 67.5
Counter({75.0: 37, 62.5: 36, 50.0: 12, 87.5: 10, 37.5: 4, 100.0: 1})
Valence 69.375
Counter({62.5: 35, 75.0: 27, 87.5: 15,

Array of average All Emotions accurecies for user s15
Counter({37.5: 31, 25.0: 28, 50.0: 23, 12.5: 7, 62.5: 6, 75.0: 3, 0.0: 2})
Combined Emotions
[37.0, 0.29920084776334777]
Arousal 44.75
Counter({37.5: 33, 50.0: 26, 62.5: 17, 25.0: 16, 75.0: 6, 12.5: 2})
Valence 60.375
Counter({62.5: 35, 75.0: 19, 50.0: 19, 37.5: 11, 87.5: 10, 25.0: 3, 12.5: 1, 100.0: 1, 0.0: 1})
Arousal Valence
[44.75, 60.375, 0.4141172161172162, 0.5822616272616272]
s16
Array of average Happy Sad accurecies for user s16
Counter({80.0: 37, 60.0: 25, 40.0: 19, 100.0: 18, 0.0: 1})
Happy Sad
[70.2, 0.6669761904761904]
Array of average All Emotions accurecies for user s16
Counter({37.5: 25, 25.0: 22, 50.0: 20, 12.5: 12, 62.5: 9, 0.0: 6, 75.0: 5, 87.5: 1})
Combined Emotions
[36.625, 0.3166488095238095]
Arousal 64.5
Counter({62.5: 29, 75.0: 29, 50.0: 17, 87.5: 13, 37.5: 8, 25.0: 3, 100.0: 1})
Valence 60.125
Counter({62.5: 39, 50.0: 23, 75.0: 21, 37.5: 10, 87.5: 5, 12.5: 1, 25.0: 1})
Arousal Valence
[64.5, 60.125, 0.6418538

Array of average Happy Sad accurecies for user s29
Counter({66.66666666666666: 34, 83.33333333333334: 29, 100.0: 20, 50.0: 11, 33.33333333333333: 5, 16.666666666666664: 1})
Happy Sad
[74.16666666666667, 0.678041366041366]
Array of average All Emotions accurecies for user s29
Counter({50.0: 25, 62.5: 24, 75.0: 19, 37.5: 16, 25.0: 9, 87.5: 5, 100.0: 2})
Combined Emotions
[56.375, 0.45254920079920075]
Arousal 72.875
Counter({75.0: 32, 87.5: 27, 62.5: 17, 50.0: 15, 100.0: 6, 37.5: 3})
Valence 56.625
Counter({62.5: 28, 37.5: 21, 75.0: 20, 50.0: 19, 25.0: 6, 87.5: 6})
Arousal Valence
[72.875, 56.625, 0.6506280386280384, 0.5418825618825618]
s30
Array of average Happy Sad accurecies for user s30
Counter({71.42857142857143: 37, 57.14285714285714: 24, 85.71428571428571: 23, 42.857142857142854: 11, 28.57142857142857: 3, 100.0: 2})
Happy Sad
[67.42857142857142, 0.5678201798201796]
Array of average All Emotions accurecies for user s30
Counter({50.0: 32, 62.5: 31, 75.0: 19, 37.5: 13, 87.5: 4, 25.0: 